In [1]:
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers import Input, Embedding, LSTM, Dense, concatenate, Dropout
from keras.models import Model


import pandas as pd
import numpy as np

import pickle
from tqdm import tqdm
import os


import warnings
warnings.filterwarnings("ignore")

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

Using TensorFlow backend.


In [2]:
data=pd.read_csv("fully_processed_data.csv")
data.head(3)

,Unnamed: 0,school_state,teacher_prefix,project_grade_category,teacher_number_of_previously_posted_projects,project_is_approved,clean_categories,clean_subcategories,essay,project_title,price,std_price,nrm_price,presence_of_the_numerical_digits,quantity
0,0,in,mrs,grades_prek_2,0,0,literacy language,esl literacy,students english learners working english seco...,educational support english learners home,154.60,-0.390533,0.015397,0,23
1,1,fl,mr,grades_6_8,7,1,history civics health sports,civics government team sports,students arrive school eager learn polite gene...,wanted projector hungry learners,299.00,0.002396,0.029839,0,1
2,2,az,ms,grades_6_8,1,0,health sports,health wellness team sports,true champions not always ones win guts mia ha...,soccer equipment awesome middle school students,516.85,0.595191,0.051628,0,22


In [3]:
from sklearn.model_selection import train_test_split

X=data.drop('project_is_approved', axis=1)
y=data['project_is_approved']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=5)

In [4]:
n_train=X_train.shape[0]
n_test=X_test.shape[0]

In [5]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer

encoder=OneHotEncoder().fit(X_train['school_state'].values.reshape(n_train,1))
enc_school_state_train=pd.DataFrame(encoder.transform(X_train['school_state'].values.reshape(n_train,1)).todense(),columns=["school_state"]*X_train['school_state'].nunique())
enc_school_state_test=pd.DataFrame(encoder.transform(X_test['school_state'].values.reshape(n_test,1)).todense(),columns=["school_state"]*X_train['school_state'].nunique())

encoder=OneHotEncoder().fit(X_train['teacher_prefix'].values.reshape(n_train,1))
enc_teacher_prefix_train=pd.DataFrame(encoder.transform(X_train['teacher_prefix'].values.reshape(n_train,1)).todense(),columns=["teacher_prefix"]*X_train['teacher_prefix'].nunique())
enc_teacher_prefix_test=pd.DataFrame(encoder.transform(X_test['teacher_prefix'].values.reshape(n_test,1)).todense(),columns=["teacher_prefix"]*X_train['teacher_prefix'].nunique())

encoder=OneHotEncoder().fit(X_train['project_grade_category'].values.reshape(n_train,1))
enc_project_grade_category_train=pd.DataFrame(encoder.transform(X_train['project_grade_category'].values.reshape(n_train,1)).todense(),columns=["project_grade_category"]*X_train['project_grade_category'].nunique())
enc_project_grade_category_test=pd.DataFrame(encoder.transform(X_test['project_grade_category'].values.reshape(n_test,1)).todense(),columns=["project_grade_category"]*X_train['project_grade_category'].nunique())

vectorizer1=CountVectorizer(binary=True)
enc_clean_categories_train=pd.DataFrame(vectorizer1.fit_transform(X_train['clean_categories']).todense(),columns=vectorizer1.get_feature_names())
enc_clean_categories_test=pd.DataFrame(vectorizer1.transform(X_test['clean_categories']).todense(),columns=vectorizer1.get_feature_names())

vectorizer2=CountVectorizer(binary=True)
enc_clean_subcategories_train=pd.DataFrame(vectorizer2.fit_transform(X_train['clean_subcategories']).todense(),columns=vectorizer2.get_feature_names())
enc_clean_subcategories_test=pd.DataFrame(vectorizer2.transform(X_test['clean_subcategories']).todense(),columns=vectorizer2.get_feature_names())

In [6]:
encoded_df_train=pd.DataFrame()
encoded_df_train=pd.concat([enc_school_state_train,enc_teacher_prefix_train,enc_project_grade_category_train,enc_clean_categories_train,enc_clean_subcategories_train], axis=1)

encoded_df_test=pd.DataFrame()
encoded_df_test=pd.concat([enc_school_state_test,enc_teacher_prefix_test,enc_project_grade_category_test,enc_clean_categories_test,enc_clean_subcategories_test], axis=1)

In [7]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

encoded_df_train['nrm_teacher_number_of_previously_posted_projects']=scaler.fit_transform(X_train['teacher_number_of_previously_posted_projects'].values.reshape(-1, 1))
encoded_df_test['nrm_teacher_number_of_previously_posted_projects']=scaler.transform(X_test['teacher_number_of_previously_posted_projects'].values.reshape(-1, 1))

encoded_df_train['nrm_price']=X_train['nrm_price'].values
encoded_df_test['nrm_price']=X_test['nrm_price'].values

encoded_df_train['nrm_presence_of_the_numerical_digits']=X_train['presence_of_the_numerical_digits'].values
encoded_df_test['nrm_presence_of_the_numerical_digits']=X_test['presence_of_the_numerical_digits'].values

encoded_df_train['nrm_quantity']=scaler.fit_transform(X_train['presence_of_the_numerical_digits'].values.reshape(-1, 1))
encoded_df_test['nrm_quantity']=scaler.transform(X_test['presence_of_the_numerical_digits'].values.reshape(-1, 1))

In [8]:
encoded_df_train.head(3)

,school_state,school_state,school_state,school_state,school_state,school_state,school_state,school_state,school_state,school_state,...,sports,team,visual,warmth,wellness,writing,nrm_teacher_number_of_previously_posted_projects,nrm_price,nrm_presence_of_the_numerical_digits,nrm_quantity
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,1,-0.185953,0.014935,1,2.438385
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,-0.293475,0.000433,0,-0.410108
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0,0,0,0,0,0,-0.400996,0.058306,1,2.438385


In [9]:
encoded_df_test.head(3)

,school_state,school_state,school_state,school_state,school_state,school_state,school_state,school_state,school_state,school_state,...,sports,team,visual,warmth,wellness,writing,nrm_teacher_number_of_previously_posted_projects,nrm_price,nrm_presence_of_the_numerical_digits,nrm_quantity
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,-0.400996,0.000459,0,-0.410108
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,-0.042591,0.003934,0,-0.410108
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,-0.400996,0.022887,0,-0.410108


In [10]:
from keras.layers import Conv1D, MaxPooling2D, Input, Dense, Flatten
from keras.models import Model

conv_input = Input(shape=(129,1))
x = Conv1D(filters=64, kernel_size=4, activation='relu')(conv_input)
x = Conv1D(filters=64, kernel_size=4, activation='relu')(x)
flatten_conv_output = Flatten()(x) 

Instructions for updating:
Colocations handled automatically by placer.


## Total Text Data

### Build training data

In [11]:
docs_essay_train=list(X_train.essay.values)
labels_train=np.array(y_train)

In [12]:
#Prepare tokenizer
tokens = Tokenizer()
tokens.fit_on_texts(docs_essay_train)
vocab_size_train = len(tokens.word_index) + 1
#Integer encode the documents
encoded_docs_essay_train = tokens.texts_to_sequences(docs_essay_train)
#print(encoded_docs)
print(vocab_size_train)

51486


In [13]:
max_len=0
all_lengths=[]
for sent in docs_essay_train:
    length=len(sent.split())
    all_lengths.append(length)
print(max(all_lengths))

320


In [14]:
# pad documents to a max length of 4 words
max_length = max(all_lengths)
padded_docs_essay_train = pad_sequences(encoded_docs_essay_train, maxlen=max_length, padding='post')
print(padded_docs_essay_train)

[[  710   261    85 ...     0     0     0]
 [ 4081  1127   187 ...     0     0     0]
 [ 1480     1    26 ...     0     0     0]
 ...
 [    1  1266   812 ...     0     0     0]
 [10770   776  3097 ...     0     0     0]
 [  343    10    92 ...     0     0     0]]


In [15]:
#Load the whole embedding into memory
embeddings_index = dict()
file = open('glove.6B.300d.txt')
for line in file:
    values = line.split()
    word = values[0]
    coefs = asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
file.close()

print('Loaded %s word vectors.' % len(embeddings_index))

#Create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size_train, 300))
for word, i in tqdm(tokens.word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector #embedding_matrix.shape: (9049, 300)
        
print(len(embedding_matrix))
print(len(embedding_matrix[0]))

 37%|███▋      | 18795/51485 [00:00<00:00, 187943.46it/s]

Loaded 400000 word vectors.


100%|██████████| 51485/51485 [00:00<00:00, 214298.15it/s]

51486
300


In [16]:
#Get the flattened LSTM output for input text
input_layer1 = Input(shape=(max_length,))
embedding = Embedding(input_dim=vocab_size_train, output_dim=300, weights=[embedding_matrix], input_length=max_length, trainable=False)(input_layer1)
lstm_out = LSTM(32, return_sequences=True)(embedding)
flatten_lstm_out = Flatten()(lstm_out)

### Building test data

In [17]:
docs_essay_test=list(X_test.essay.values)
labels_test=np.array(y_test)

In [18]:
#Prepare tokenizer
tokens = Tokenizer()
tokens.fit_on_texts(docs_essay_test)
vocab_size_test = len(tokens.word_index) + 1
#Integer encode the documents
encoded_docs_essay_test = tokens.texts_to_sequences(docs_essay_test)
#print(encoded_docs)
print(vocab_size_test)

30150


In [19]:
#Pad documents to a max length
padded_docs_essay_test = pad_sequences(encoded_docs_essay_test, maxlen=max_length, padding='post')
print(padded_docs_essay_test)

[[  408   853  1683 ...     0     0     0]
 [    2    23   135 ...     0     0     0]
 [ 1401    69     1 ...     0     0     0]
 ...
 [13883 13884 13885 ...     0     0     0]
 [    1    16 15760 ...     0     0     0]
 [  466     1    83 ...     0     0     0]]


### Concatenating the layers.

In [21]:
from keras.layers import Input, Embedding, LSTM, Dense, concatenate, Dropout

x = concatenate([flatten_lstm_out, flatten_conv_output])

x = Dense(64, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(64, activation='relu')(x)

output = Dense(1, activation='sigmoid', name='output')(x)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


### Defining a custom metric AUC

In [22]:
from sklearn import metrics as sklm
import tensorflow as tf
def auc_roc(y_true, y_pred):
    try:
        return tf.py_func(sklm.roc_auc_score, (y_true, y_pred), tf.double)
    except ValueError:
        pass

### Defining a TensorBoard callback object

In [23]:
from time import time
from tensorflow.python.keras.callbacks import TensorBoard
tensorboard = TensorBoard(log_dir="logs/{}".format(time))

### Compiling the final model

In [25]:
model = Model(inputs=[input_layer1,conv_input], outputs=[output])
encoded_df_train_exp = np.expand_dims(encoded_df_train.values, axis=2)
encoded_df_test_exp = np.expand_dims(encoded_df_test.values, axis=2)

In [27]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy',auc_roc])
model.fit(x=[padded_docs_essay_train, encoded_df_train_exp], 
          y=[labels_train],
          epochs=10, 
          batch_size=64,
          validation_data=([padded_docs_essay_test, encoded_df_test_exp],[labels_test]),
          callbacks=[tensorboard])

Train on 87398 samples, validate on 21850 samples
Epoch 1/10
87398/87398 [==============================] - 581s 7ms/step - loss: 0.3969 - acc: 0.8489 - auc_roc: 0.7003 - val_loss: 0.4652 - val_acc: 0.8255 - val_auc_roc: 0.5778
Epoch 2/10
87398/87398 [==============================] - 593s 7ms/step - loss: 0.3743 - acc: 0.8529 - auc_roc: 0.7511 - val_loss: 0.4569 - val_acc: 0.8210 - val_auc_roc: 0.5741
Epoch 3/10
87398/87398 [==============================] - 596s 7ms/step - loss: 0.3487 - acc: 0.8618 - auc_roc: 0.7980 - val_loss: 0.4521 - val_acc: 0.8364 - val_auc_roc: 0.5660
Epoch 4/10
87398/87398 [==============================] - 567s 6ms/step - loss: 0.3150 - acc: 0.8759 - auc_roc: 0.8443 - val_loss: 0.4746 - val_acc: 0.8286 - val_auc_roc: 0.5636
Epoch 5/10
87398/87398 [==============================] - 563s 6ms/step - loss: 0.2758 - acc: 0.8915 - auc_roc: 0.8866 - val_loss: 0.5027 - val_acc: 0.8146 - val_auc_roc: 0.5569
Epoch 6/10
87398/87398 [==============================] - 56

In [ ]:
!tensorboard --logdir=logs/